# 第一个例子
使用ollama

# 2节点+1条边的例子

In [1]:
from langgraph.graph import END, MessageGraph
from langgraph.graph.state import CompiledStateGraph
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage

graph = MessageGraph()          #创建一个消息图
model=Ollama(model="gemma2:2b", temperature=0.3)
graph.add_node("oracle", model) #节点的名字是oracle

graph.add_edge("oracle", END)   #添加了一条边的起点就是model，终点就是END
#graph.add_conditional_edges
graph.set_entry_point("oracle") #定义了起点

runnable:CompiledStateGraph = graph.compile()      #编译图，编译后就可以直接运行了
print(runnable)
ret_list=runnable.invoke(HumanMessage('What is 1+1=?')) #ret_list=runnable.invoke('What is 1+1=?') 这样写也可以
print("\nret_list")
for t in ret_list:
    print(t)

nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<__root__>(recurse=True, writes=[ChannelWriteEntry(channel='__root__', value=<object object at 0x7026017ccb60>, skip_none=True, mapper=None)]), ChannelWrite<start:oracle>(recurse=True, writes=[ChannelWriteEntry(channel='start:oracle', value='__start__', skip_none=False, mapper=None)])]), 'oracle': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['start:oracle'], writers=[ChannelWrite<oracle,__root__>(recurse=True, writes=[ChannelWriteEntry(channel='oracle', value='oracle', skip_none=False, mapper=None), ChannelWriteEntry(channel='__root__', value=<object object at 0x7026017ccb60>, skip_none=True, mapper=None)])])} channels={'__root__': <langgraph.channels.binop.BinaryOperatorAggregate object at 0x7026083fad50>, '__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7026004fcce0>, 'oracle': <langgraph.channels.eph

# 传递参数
1. 使用custom_data从头传递到尾
2. 无法使用threading.local()实现，因为图的驱动是多线程的

In [4]:
from langgraph.graph import END, MessageGraph
from langgraph.graph.state import CompiledStateGraph
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, BaseMessage
from typing import Dict, List, Union

def node1_process(state: List[BaseMessage]) -> str:
    print(f"node1: {str(state)}")
    ret:HumanMessage=HumanMessage("node1 done")
    ret.custom_data=state[-1].custom_data
    ret.name="node1-response"
    return ret


def node2_process(state: List[BaseMessage]) -> str:
    custom_data=state[-1].custom_data
    print(f"node2: {str(state)}，custom_data={custom_data}")
    return "node2 done"


graph = MessageGraph()  # 创建一个消息图

model = Ollama(model="gemma2:2b", temperature=0.3)
graph.add_node("node1", node1_process)
graph.add_node("node2", node2_process)
graph.add_edge("node1", "node2")
graph.add_edge("node2", END)
graph.set_entry_point("node1")  # 定义了起点

runnable: CompiledStateGraph = graph.compile()  # 编译图，编译后就可以直接运行了
print(runnable)
message=HumanMessage("首发消息")
message.name="first"
message.custom_data=100
ret_list = runnable.invoke(input=[message])   # 这样写也可以ret_list = runnable.invoke(input='What is 1+1=?')

print("\nret_list")
for t in ret_list:
    print(f"[{t.type}] : {t.content}, id={t.id}")

nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<__root__>(recurse=True, writes=[ChannelWriteEntry(channel='__root__', value=<object object at 0x7d9ed0048470>, skip_none=True, mapper=None)]), ChannelWrite<start:node1>(recurse=True, writes=[ChannelWriteEntry(channel='start:node1', value='__start__', skip_none=False, mapper=None)])]), 'node1': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['start:node1'], writers=[ChannelWrite<node1,__root__>(recurse=True, writes=[ChannelWriteEntry(channel='node1', value='node1', skip_none=False, mapper=None), ChannelWriteEntry(channel='__root__', value=<object object at 0x7d9ed0048470>, skip_none=True, mapper=None)])]), 'node2': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['node1'], writers=[ChannelWrite<node2,__root__>(recurse=True, writes=[ChannelWriteEntry(channel='node2', value='node2', skip_none=False, mapper=None), Chann

# 把chain实例作为节点的例子
1. 演示了多次大模型调用，后面的大模型请求时需要用到前一次的处理结果


In [2]:
import logging
import os
from typing import List, Union

from langgraph.graph import END, MessageGraph
from langgraph.graph.state import CompiledStateGraph
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#os.environ["LANGCHAIN_TRACING"]="true"  #配置了会向langsmith发送数据

logging.basicConfig(level=logging.INFO)
def extract_content_func1(inputs: str):
    return inputs.strip()

graph = MessageGraph()
llm_model = Ollama(model="gemma2:9b", temperature=0.3, verbose=True)
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "从语句中提取动词，要求返回中文"),
    MessagesPlaceholder(variable_name="name"),
])

chain1 = prompt1 | llm_model | extract_content_func1

def extract_content_func2(inputs:Union[List[BaseMessage], str]):
    if isinstance(inputs, List):
        return str(inputs[-1]).strip()
    elif isinstance(inputs, str):
        return inputs.strip()
    else:
        raise Exception(f"extract_content_func2不支持此类型的输入[{type(inputs)}]")


def node2_process(inputs: List[BaseMessage]):
    message_list = [SystemMessage(content="你是一个专业翻译，现在请你把给你的汉字都翻译成英文", name="system"),
           HumanMessage(content= inputs[-1].content, name="human")
          ]
    response=llm_model.invoke(message_list)
    ret=extract_content_func2(response)
    return ret


def node3_process(inputs: List[BaseMessage]):
    print(inputs[-1].content)

graph.add_node("node1", chain1)         #节点的名字是node1，添加的是一个链
graph.add_node("node2", node2_process)  #节点node2的作用是接受node1的输出，并继续调用大模型翻译成英文
graph.add_node("node3", node3_process)  #打印输出结果
graph.add_edge("node1", "node2")
graph.add_edge("node2", "node3")
graph.add_edge("node3", END)

graph.set_entry_point("node1")

runnable: CompiledStateGraph = graph.compile()
ret_list = runnable.invoke(HumanMessage(["打开电脑"]))
print("\nret_list is")
for t in ret_list:
    print(t)


Open

ret_list is
content=['打开电脑'] id='e989db82-c837-4afb-99b1-c08f38971a43'
content='打开' id='8b9e0703-56fc-4de1-9c55-a8ea12866cae'
content='Open' id='01808ec2-8dc0-406f-9f39-15ed8ef1f9f4'


# 添加了条件边的例子

In [3]:
from typing import List, Union

from langgraph.graph import END, MessageGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def print_message(message:BaseMessage)->None:
    print(f"\ttype={message.type},name={message.name},content={message.content},id={message.id}, additional_kwargs={message.additional_kwargs}")


print()
if __name__=="__main__":
    def after_model_process(val:str)->Union[str, int, float, bool, BaseMessage]:
        """
        llm执行后的处理函数，主要用来从结果中提取数据
        """
        return AIMessage(name="ai", content=val[val.rfind("**")+2:])

    graph = MessageGraph()          #创建一个消息图
    model=Ollama(model="gemma:7b", temperature=0.3)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "从语句中提取动词，要求返回中文，并且不要带标点符号"),
        MessagesPlaceholder(variable_name="name")])
    prompt.name="prompt1"
    chain = prompt | model | after_model_process

    def node0_process(state:List[BaseMessage]):
        """
        节点0的处理函数
        :param state:
        :return:
        """
        state[-1].data=100
        pass

    def node1_process(state: List[BaseMessage]):
        print("node1节点不会被执行")

    def node2_process(state: List[BaseMessage]):
        print("执行node2节点")
        for item in state: print_message(item)

    def edge_route1(state:List[BaseMessage])->str:
        """
        条件边的路由函数，返回的字符串必须和add_condition_edges添加的condition_edge_mapping的key保持一致
        :param state:
        :return:
        """
        return "节点2"

    graph.add_node("node0", node0_process)
    graph.add_node("oracle", chain)         #节点的名字是oracle
    graph.add_node("node1", node1_process)  #添加了一个node1节点
    graph.add_node("node2", node2_process)  # 添加了一个node2节点
    graph.add_conditional_edges("oracle", edge_route1,  {
        "节点1": "node1",     #key是名字，值是可选的返回节点名字
        "节点2": "node2"
    })   #oracle->node1 | node2
    graph.add_edge("node0", "oracle")   #node0->oracle
    graph.add_edge("node1", END)   #node1->END
    graph.add_edge("node2", END)  # node2->END
    graph.set_entry_point("node0") #定义了起点

    runnable:CompiledGraph = graph.compile()      #编译图，编译后就可以直接运行了
    message_list:BaseMessage=HumanMessage(name="human", content="打开电脑")
    ret_list=runnable.invoke(message_list)
    print("\nret_list")
    for t in ret_list:
        print_message(t)



执行node2节点
	type=human,name=human,content=打开电脑,id=54d5b47e-90fb-4dfa-bd0c-3e43250a0abe, additional_kwargs={}
	type=ai,name=ai,content=开放电脑。,id=6ab9e8df-09ff-4fdd-856d-275011402ec9, additional_kwargs={}

ret_list
	type=human,name=human,content=打开电脑,id=54d5b47e-90fb-4dfa-bd0c-3e43250a0abe, additional_kwargs={}
	type=ai,name=ai,content=开放电脑。,id=6ab9e8df-09ff-4fdd-856d-275011402ec9, additional_kwargs={}


In [ ]:
import re

text = '**系统：** 从语句“打开电脑”中提取动词，返回中文，结果为“打开”。'

def extract_string(text):
    pattern=r'[“"]([^”"]*)[”"]\s*'
    extracted_string = re.findall(pattern, text)
    return extracted_string[-1]

# 示例
extracted_string = extract_string(text)
print(extracted_string)


# 循环执行节点
传入一段话提取其中的魔法数字，然后找出1与魔法数字之间的奇数，最终输出奇数的数量
1. 大模型调用前后都加了事件方法
2. 增加了条件边
3. 增加了条件循环


In [4]:
from typing import List, Union, Dict, Any, Optional
from langchain_core.prompt_values import ChatPromptValue
from langgraph.graph import END, MessageGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import random
import sys

def print_message(message:BaseMessage)->None:
    print(f"\ttype={message.type},name={message.name},content={message.content},id={message.id}, additional_kwargs={message.additional_kwargs}")

print()
PREFIX="答案："
if __name__=="__main__":
    sys.setrecursionlimit(5000) #设置全局递归上限，太小了执行会报错

    def before_oracle(val:ChatPromptValue)->ChatPromptValue:
        """
        llm执行前的事件
        """
        return val

    def after_oracle(val:str)->Union[str, int, float, bool, BaseMessage]:
        """
        llm执行后的处理函数，主要用来从结果中提取数据
        """
        temp=val[len(PREFIX):val.rfind("\n\n")]
        ret:AIMessage=AIMessage(name="ai", content=val)
        ret.additional_kwargs['value']=int(temp)
        ret.additional_kwargs['odd_count']=0
        return ret

    graph = MessageGraph()          #创建一个消息图
    model=Ollama(model="gemma:7b", temperature=0.3)
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
##目标
请帮我提取这个魔法数字，回答保持极简

##输出
{PREFIX}Number

##注意事项
只能输出魔法数字，不要出现别的文字
         """),
        MessagesPlaceholder(variable_name="name")])
    prompt.name="prompt1"
    chain = prompt | before_oracle | model | after_oracle

    def init_process(state:List[BaseMessage]):
        """
        节点初始化处理函数
        """
        humanMessage:HumanMessage=state[0] # type: ignore
        assert isinstance(humanMessage, HumanMessage)
        humanMessage.additional_kwargs['loop_count']=1

    def node_invoke_process(state:List[BaseMessage]):
        """
        执行节点，负责执奇数判定和计数器
        """
        humanMessage:HumanMessage=state[0] # type: ignore
        assert isinstance(humanMessage, HumanMessage)

        aiMessage:AIMessage=state[1] # type: ignore
        assert isinstance(aiMessage, AIMessage)
        is_odd:bool=aiMessage.additional_kwargs['value'] % 2==1
        if is_odd:
            aiMessage.additional_kwargs['odd_count']+=1
        print(f"执行invoke node节点第{humanMessage.additional_kwargs['loop_count']}次，当前值:{aiMessage.additional_kwargs['value']}"+(" 是奇数" if is_odd else ""))
        aiMessage.additional_kwargs['value']-=1
        humanMessage.additional_kwargs['loop_count']+=1

    def edge_invoke_route(state:List[BaseMessage])->str:
        """
        校验边，负责决定结束条件，如果不满足条件就退回
        """
        aiMessage:AIMessage=state[1] # type: ignore
        assert isinstance(aiMessage, AIMessage)
        if aiMessage.additional_kwargs['value']==0:
            return "输出结论"
        return "节点invoke"

    def node_conclusion_process(state: List[BaseMessage]):
        """
        节点-输出结论
        """
        humanMessage:HumanMessage=state[0] # type: ignore
        assert isinstance(humanMessage, HumanMessage)

        aiMessage:AIMessage=state[1] # type: ignore
        assert isinstance(aiMessage, AIMessage)

        print(f"结论：总共执行了{humanMessage.additional_kwargs['loop_count']}次，奇数有{aiMessage.additional_kwargs['odd_count']}个")

    graph.add_node("node_init", init_process)                   #添加一个初始化节点
    graph.add_node("oracle", chain)
    graph.add_node("node_invoke", node_invoke_process)          #添加一个执行节点
    graph.add_node("node_conclusion", node_conclusion_process)  #添加了一个结论节点

    graph.add_conditional_edges("node_invoke", edge_invoke_route,  {
        "节点invoke": "node_invoke",        #key是名字，值是可选的返回节点名字
        "输出结论": "node_conclusion"
    })                                      #条件边：初始化节点可以到执行节点也可以到结论节点
    graph.add_edge("node_init", "oracle")   #初始化节点到大模型节点
    graph.add_edge("oracle", "node_invoke") #大模型节点到执行节点
    graph.add_edge("node_conclusion", END)  #结论节点到结束节点
    graph.set_entry_point("node_init")      #定义了起点

    runnable:CompiledGraph = graph.compile()      #编译图，编译后就可以直接运行了
    message_list:BaseMessage=HumanMessage(name="human", content=f"我给出的魔法数字是{random.randint(5,100)}，这个数字有魔力的哦")
    ret_list:List[BaseMessage]=runnable.invoke(message_list, config={
        "recursion_limit": 3000,    #设置LangChain的递归上限值，太小了会抛出GraphRecursionError异常
    }) #type: ignore
    print("\n执行结束，返回结果")
    for t in ret_list:
        print_message(t)



执行invoke node节点第1次，当前值:21 是奇数
执行invoke node节点第2次，当前值:20
执行invoke node节点第3次，当前值:19 是奇数
执行invoke node节点第4次，当前值:18
执行invoke node节点第5次，当前值:17 是奇数
执行invoke node节点第6次，当前值:16
执行invoke node节点第7次，当前值:15 是奇数
执行invoke node节点第8次，当前值:14
执行invoke node节点第9次，当前值:13 是奇数
执行invoke node节点第10次，当前值:12
执行invoke node节点第11次，当前值:11 是奇数
执行invoke node节点第12次，当前值:10
执行invoke node节点第13次，当前值:9 是奇数
执行invoke node节点第14次，当前值:8
执行invoke node节点第15次，当前值:7 是奇数
执行invoke node节点第16次，当前值:6
执行invoke node节点第17次，当前值:5 是奇数
执行invoke node节点第18次，当前值:4
执行invoke node节点第19次，当前值:3 是奇数
执行invoke node节点第20次，当前值:2
执行invoke node节点第21次，当前值:1 是奇数
结论：总共执行了22次，奇数有11个

执行结束，返回结果
	type=human,name=human,content=我给出的魔法数字是21，这个数字有魔力的哦,id=7c8cfa5c-f9e7-46b5-af43-bbb2a29da584, additional_kwargs={'loop_count': 22}
	type=ai,name=ai,content=答案：21

魔法数字是21，根据人说的。,id=e7ebb7ea-765d-4022-8fa4-93f72b0d9777, additional_kwargs={'value': 0, 'odd_count': 11}


# 使用StateGraph的例子

In [7]:
from typing import List, TypedDict, Annotated, Sequence, Union, Dict, Any, Optional
import operator
from langchain_core.prompt_values import ChatPromptValue
from langgraph.graph import END, StateGraph, MessageGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import random
import sys


def print_message(message: BaseMessage) -> None:
    print(f"\ttype={message.type},name={message.name},content={message.content},id={message.id}, additional_kwargs={message.additional_kwargs}")


class AgentState(TypedDict):
    """
    定义Agent状态类，其中重载了+操作符，实现了通过+来向AgentState里添加消息
    """
    messages: Annotated[Sequence[BaseMessage], operator.add]


print()
PREFIX = "答案："
if __name__ == "__main__":
    sys.setrecursionlimit(5000)  # 设置全局递归上限，太小了执行会报错


    def before_oracle(val: ChatPromptValue) -> ChatPromptValue:
        """
        llm执行前的事件
        """
        return val


    def after_oracle(val: str) -> Union[str, int, float, bool, BaseMessage]:
        """
        llm执行后的处理函数，主要用来从结果中提取数据
        """
        temp = val[len(PREFIX):val.rfind("\n\n")]
        ret: AIMessage = AIMessage(name="ai", content=val)
        ret.additional_kwargs['value'] = int(temp)
        ret.additional_kwargs['odd_count'] = 0
        return ret

    graph = MessageGraph()  # 创建一个状态图
    model = Ollama(model="gemma2:2b", temperature=0.3)
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
##目标
请帮我提取这个魔法数字，回答保持极简

##输出
{PREFIX}

##注意事项
只能输出魔法数字，不要出现别的文字
         """),
        MessagesPlaceholder(variable_name="name")])
    prompt.name = "prompt1"
    chain = prompt | before_oracle | model | after_oracle


    def init_process(state: List[BaseMessage]):
        """
        节点初始化处理函数
        """
        humanMessage: HumanMessage = state[0]  # type: ignore
        assert isinstance(humanMessage, HumanMessage)
        humanMessage.additional_kwargs['loop_count'] = 1


    def node_invoke_process(state: List[BaseMessage]):
        """
        执行节点，负责执奇数判定和计数器
        """
        humanMessage: HumanMessage = [item for item in state if item.type=='human'][0]  # type: ignore
        assert isinstance(humanMessage, HumanMessage)

        aiMessage: AIMessage = [item for item in state if item.type=='ai'][0]  # type: ignore
        assert isinstance(aiMessage, AIMessage)
        is_odd: bool = aiMessage.additional_kwargs['value'] % 2 == 1
        if is_odd:
            aiMessage.additional_kwargs['odd_count'] += 1
        print(f"执行invoke node节点第{humanMessage.additional_kwargs['loop_count']}次，当前值:{aiMessage.additional_kwargs['value']}" + (" 是奇数" if is_odd else ""))
        aiMessage.additional_kwargs['value'] -= 1
        humanMessage.additional_kwargs['loop_count'] += 1


    def edge_invoke_route(state: List[BaseMessage]) -> str:
        """
        校验边，负责决定结束条件，如果不满足条件就退回
        """
        aiMessage: AIMessage = [item for item in state if item.type=='ai'][0]  # type: ignore
        assert isinstance(aiMessage, AIMessage)
        if aiMessage.additional_kwargs['value'] == 0:
            return "输出结论"
        return "节点invoke"


    def node_conclusion_process(state: List[BaseMessage]):
        """
        节点-输出结论
        """
        humanMessage: HumanMessage = [item for item in state if item.type=='human'][0]  # type: ignore
        assert isinstance(humanMessage, HumanMessage)

        aiMessage: AIMessage = [item for item in state if item.type=='ai'][0]  # type: ignore
        assert isinstance(aiMessage, AIMessage)

        print(f"结论：总共执行了{humanMessage.additional_kwargs['loop_count']}次，奇数有{aiMessage.additional_kwargs['odd_count']}个")


    graph.add_node("node_init", init_process)  # 添加一个初始化节点
    graph.add_node("oracle", chain)
    graph.add_node("node_invoke", node_invoke_process)  # 添加一个执行节点
    graph.add_node("node_conclusion", node_conclusion_process)  # 添加了一个结论节点

    graph.add_conditional_edges("node_invoke", edge_invoke_route, {
        "节点invoke": "node_invoke",  # key是名字，值是可选的返回节点名字
        "输出结论": "node_conclusion"
    })  # 条件边：初始化节点可以到执行节点也可以到结论节点
    graph.add_edge("node_init", "oracle")  # 初始化节点到大模型节点
    graph.add_edge("oracle", "node_invoke")  # 大模型节点到执行节点
    graph.add_edge("node_conclusion", END)  # 结论节点到结束节点
    graph.set_entry_point("node_init")  # 定义了起点

    runnable: CompiledGraph = graph.compile()  # 编译图，编译后就可以直接运行了
    # message_list:BaseMessage=HumanMessage(name="human", content=f"我给出的魔法数字是{random.randint(5,100)}，这个数字有魔力的哦")

    message_list: List[BaseMessage] = [
        HumanMessage(name="human", content=f"我给出的魔法数字是{random.randint(5, 100)}，这个数字有魔力的哦"),
        SystemMessage(name="system", content=f"你是一个数学家，擅长处理各种数字")
    ]

    ret_list: List[BaseMessage] = runnable.invoke(message_list, config={
        "recursion_limit": 3000,  # 设置LangChain的递归上限值，太小了会抛出GraphRecursionError异常
    })  # type: ignore
    print("\n执行结束，返回结果")
    for t in ret_list:
        print_message(t)



执行invoke node节点第1次，当前值:28
执行invoke node节点第2次，当前值:27 是奇数
执行invoke node节点第3次，当前值:26
执行invoke node节点第4次，当前值:25 是奇数
执行invoke node节点第5次，当前值:24
执行invoke node节点第6次，当前值:23 是奇数
执行invoke node节点第7次，当前值:22
执行invoke node节点第8次，当前值:21 是奇数
执行invoke node节点第9次，当前值:20
执行invoke node节点第10次，当前值:19 是奇数
执行invoke node节点第11次，当前值:18
执行invoke node节点第12次，当前值:17 是奇数
执行invoke node节点第13次，当前值:16
执行invoke node节点第14次，当前值:15 是奇数
执行invoke node节点第15次，当前值:14
执行invoke node节点第16次，当前值:13 是奇数
执行invoke node节点第17次，当前值:12
执行invoke node节点第18次，当前值:11 是奇数
执行invoke node节点第19次，当前值:10
执行invoke node节点第20次，当前值:9 是奇数
执行invoke node节点第21次，当前值:8
执行invoke node节点第22次，当前值:7 是奇数
执行invoke node节点第23次，当前值:6
执行invoke node节点第24次，当前值:5 是奇数
执行invoke node节点第25次，当前值:4
执行invoke node节点第26次，当前值:3 是奇数
执行invoke node节点第27次，当前值:2
执行invoke node节点第28次，当前值:1 是奇数
结论：总共执行了29次，奇数有14个

执行结束，返回结果
	type=human,name=human,content=我给出的魔法数字是28，这个数字有魔力的哦,id=1059f3c2-801c-405b-b6fb-c893e260f1f5, additional_kwargs={'loop_count': 29}
	type=system,name=system,content=你是一个数学家，擅长处